# Librerías

In [1]:
import oracledb
import os 
from pathlib import Path
from datetime import datetime
from typing import Optional

# Conexión reutilizable

In [2]:
def get_connection():
    return oracledb.connect(
        user="SYSTEM",
        password="96443804",
        dsn="localhost:1521/xe"
    ) 

# Esquema base de datos

In [3]:
def create_schema(query):
    try:         
        with get_connection() as conn:             
            with conn.cursor() as cur:                 
                cur.execute(query)                 
                print(f'✓ Tabla creada exitosamente')
            conn.commit()     
    except oracledb.DatabaseError as e:        
        err = e
        if 'ORA-00955' in str(err):  # Tabla ya existe
            pass  # No imprimir nada si la tabla ya existe
        else:
            print(f"Error al crear tabla: {err}")

# Query con la creación de tablas

In [4]:
def create_all_tables():
    tables = [
        (
             "CREATE TABLE "
             "PERSONAS ("
             "RUT VARCHAR(10) PRIMARY KEY,"
             "Nombres VARCHAR(64),"
             "Apellidos VARCHAR(64)"
             ")"
        ),
        (
             "CREATE TABLE "
             "ASIGNATURA ("
             "ID INTEGER PRIMARY KEY,"
             "Nombre VARCHAR(32)"
             ")"
        ),
        (
             "CREATE TABLE "
             "PROFESORES ("
             "ID INTEGER PRIMARY KEY,"
             "PROFESOR VARCHAR(10),"
             "ASIGNATURA INTEGER,"
             "FOREIGN KEY (PROFESOR) REFERENCES PERSONAS(RUT),"
             "FOREIGN KEY (ASIGNATURA) REFERENCES ASIGNATURA(ID)"
             ")"
        ),
        (
            "CREATE TABLE "
            "CURSOS ("
            "ID VARCHAR(10) PRIMARY KEY,"
            "ALUMNO VARCHAR(10),"
            "FOREIGN KEY (ALUMNO) REFERENCES PERSONAS(RUT)"
            ")"
        ),
        (
            "CREATE TABLE "
            "HORARIOS ("
            "ID INTEGER PRIMARY KEY,"
            "PROFESOR INTEGER,"
            "CURSO VARCHAR(10),"
            "FOREIGN KEY (PROFESOR) REFERENCES PROFESORES(ID),"
            "FOREIGN KEY (CURSO) REFERENCES CURSOS(ID)"
            ")"
        ),
        (
            "CREATE TABLE "
            "EVALUACIONES ("
            "ID INTEGER PRIMARY KEY,"
            "NOTA INTEGER,"
            "ASIGNATURA INTEGER,"
            "ALUMNO VARCHAR(10),"
            "PROFESOR INTEGER,"
            "FOREIGN KEY (ASIGNATURA) REFERENCES ASIGNATURA(ID),"
            "FOREIGN KEY (ALUMNO) REFERENCES PERSONAS(RUT),"
            "FOREIGN KEY (PROFESOR) REFERENCES PROFESORES(ID)"
            ")"
        )
    ]
    for query in tables:
        create_schema(query)


# Script para limpiar tablas previamente creadas

In [15]:
try:
    conn = oracledb.connect(user="SYSTEM", password="96443804", dsn="localhost:1521/xe")
    cur = conn.cursor()
    
    # Orden de eliminación (respetando foreign keys)
    tablas = ['EVALUACIONES', 'HORARIOS', 'CURSOS', 'PROFESORES', 'ASIGNATURA', 'PERSONAS']
    
    for tabla in tablas:
        try:
            cur.execute(f'DROP TABLE {tabla}')
            conn.commit()
            print(f'✓ Tabla {tabla} eliminada')
        except oracledb.DatabaseError as e:
            if 'ORA-00942' in str(e):  # Tabla no existe
                print(f'- Tabla {tabla} no existe (ignorado)')
            else:
                print(f'✗ Error al eliminar {tabla}: {e}')
    
    conn.close()
    print("\n✓ Limpieza completada exitosamente")
    
except oracledb.DatabaseError as e:
    print(f"Error de conexión: {e}")

✓ Tabla EVALUACIONES eliminada
✓ Tabla HORARIOS eliminada
✓ Tabla CURSOS eliminada
✓ Tabla PROFESORES eliminada
✓ Tabla ASIGNATURA eliminada
✓ Tabla PERSONAS eliminada

✓ Limpieza completada exitosamente
✓ Tabla PROFESORES eliminada
✓ Tabla ASIGNATURA eliminada
✓ Tabla PERSONAS eliminada

✓ Limpieza completada exitosamente


# Creación de tablas

In [16]:
create_all_tables()

✓ Tabla creada exitosamente
✓ Tabla creada exitosamente
✓ Tabla creada exitosamente
✓ Tabla creada exitosamente
✓ Tabla creada exitosamente
✓ Tabla creada exitosamente
✓ Tabla creada exitosamente
✓ Tabla creada exitosamente
✓ Tabla creada exitosamente
✓ Tabla creada exitosamente


# Funciones par insertar datos

## Tabla persona

In [7]:
def create_persona(rut=None, nombres=None, apellidos=None):     
    sql = (        
    "INSERT INTO personas (rut, nombres, apellidos) "         
    "VALUES (:rut, :nombres, :apellidos)"     
    )     
    try:
        with get_connection() as conn:         
            with conn.cursor() as cur:             
                cur.execute(sql, {                 
                "rut": rut,                 
                "nombres": nombres,                 
                "apellidos": apellidos             
                })             
            conn.commit()            
        print(f"✓ Persona con RUT={rut} creada.")
    except oracledb.DatabaseError as e:
        print(f"Error al crear persona: {e}") 

## Tabla asignatura

In [8]:
def create_asignatura(id=None, nombre=None):     
    sql = (        
    "INSERT INTO ASIGNATURA (id, nombre)"         
    "VALUES (:id, :nombre)"     
    )    
    with get_connection() as conn:         
        with conn.cursor() as cur:             
            cur.execute(sql, {                 
            "id": id,                 
            "nombre": nombre          
            })             
        conn.commit()            
    print(f"Departamento {nombre} creado.") 

## Tabla Profesores

In [9]:
def create_profesores(ID=None, PROFESOR=None, ASIGNATURA=None):     
    sql = (        
    "INSERT INTO PROFESORES (ID, PROFESOR, ASIGNATURA)"         
    "VALUES (:ID, :PROFESOR, :ASIGNATURA)"     
    )    
    with get_connection() as conn:         
        with conn.cursor() as cur:             
            cur.execute(sql, {                 
            "ID": ID,
            "PROFESOR": PROFESOR,                 
            "ASIGNATURA": ASIGNATURA        
            })             
        conn.commit()            
    print(f"Profesor {PROFESOR} registrado.") 

## Tabla Cursos

In [10]:
def create_cursos(ID=None, ALUMNO=None):     
    sql = (        
    "INSERT INTO CURSOS (ID, ALUMNO)"         
    "VALUES (:ID, :ALUMNO)"     
    )    
    with get_connection() as conn:         
        with conn.cursor() as cur:             
            cur.execute(sql, {                 
            "ID": ID,                 
            "ALUMNO": ALUMNO        
            })             
        conn.commit()            
    print(f"Alumno {ALUMNO} registrado en curso {ID}.")     


## Tabla Horarios

In [11]:
def create_horarios(ID=None, PROFESOR=None, CURSO=None):     
    sql = (        
    "INSERT INTO HORARIOS (ID, PROFESOR, CURSO)"         
    "VALUES (:ID, :PROFESOR, :CURSO)"     
    )    
    with get_connection() as conn:         
        with conn.cursor() as cur:             
            cur.execute(sql, {                 
            "ID": ID,                 
            "PROFESOR": PROFESOR,
            "CURSO": CURSO    
            })             
        conn.commit()            
    print(f"Profesor {PROFESOR} dictará curso {CURSO}.")    


## Tabla Evaluaciones

In [12]:
def create_evaluaciones(ID=None, NOTA=None, ASIGNATURA=None, ALUMNO=None, PROFESOR=None):     
    sql = (        
    "INSERT INTO EVALUACIONES (ID, NOTA, ASIGNATURA, ALUMNO, PROFESOR)"         
    "VALUES (:ID, :NOTA, :ASIGNATURA, :ALUMNO, :PROFESOR)"     
    )    
    with get_connection() as conn:         
        with conn.cursor() as cur:             
            cur.execute(sql, {                 
            "ID": ID,
            "NOTA": NOTA,
            "ASIGNATURA": ASIGNATURA,
            "ALUMNO": ALUMNO,                 
            "PROFESOR": PROFESOR 
            })             
        conn.commit()            
    print(f"Alumno {ALUMNO} recibió la nota {NOTA}.") 


# Funciones para leer tablas

## Leer todos los datos de una tabla

In [13]:
def read_table(tabla):
    sql = (
        f"SELECT * FROM {tabla}"
    )
    try:
        with get_connection() as conn:
            with conn.cursor() as cur:
                resultados = cur.execute(sql)
                print(f"Consulta a la tabla {tabla}")
                for fila in resultados:
                    print(fila)
    except oracledb.DatabaseError as e:
        err = e
        print(f"Error al leer datos: {err}")

## Buscar por ID o RUT

In [14]:
def read_table_by_id(tabla, columna_id, valor_id):
    sql = (
        f"SELECT * FROM {tabla} WHERE {columna_id} = :valor"
    )
    parametros = {"valor": valor_id}
    try:
        with get_connection() as conn:
            with conn.cursor() as cur:
                resultados = cur.execute(sql, parametros)
                print(f"✓ Consulta a la tabla {tabla} donde {columna_id} = {valor_id}")
                encontrados = 0
                for fila in resultados:
                    print(f"  {fila}")
                    encontrados += 1
                if encontrados == 0:
                    print(f"  No se encontraron registros")
    except oracledb.DatabaseError as e:
        print(f"Error al leer datos: {e}")